In [1]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


alberto-lorente


In [3]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = torch.device('cuda')

Cuda available


In [4]:
from council_rag.preprocessing import preprocess_markdown_text

In [5]:
paragraphs, clusters = preprocess_markdown_text(markdown_example,
                                                    model_id ="HIT-TMG/KaLM-embedding-multilingual-mini-v1", 
                                                    spacy_model="fr_core_news_sm", 
                                                    n_sents_per_para=10)

Cuda available
Number of clusters:  3
Score:  0.44600135

Number of clusters:  4
Score:  0.44329956

Number of clusters:  5
Score:  0.47113436

Number of clusters:  6
Score:  0.48168555

Number of clusters:  7
Score:  0.48469815

Number of clusters:  8
Score:  0.5539949

Index 5
Optimal Number of Clusters 8


In [7]:
paragraphs[0].keys()

dict_keys(['paragraph_union', 'start_range', 'stop_range', 'list_sents', 'para_embedding', 'para_cluster'])

In [8]:
paragraphs[0]

{'paragraph_union': '# Séance publique\n\n\n\n # du jeudi 28 octobre 2021\n\n\n\n # à 18h00\n\n\n\n# Chorum Alain Gilles - Halle Vacheresse\n\n\n\n # Rue des Vernes à Roanne\n\n\n\n #',
 'start_range': np.int64(0),
 'stop_range': np.int64(10),
 'list_sents': ['#',
  'Séance publique\n\n\n\n',
  '# du jeudi 28 octobre 2021\n\n\n\n',
  '# à 18h00\n\n\n\n#',
  'Chorum',
  'Alain Gilles',
  '- Halle Vacheresse\n\n\n\n',
  '#',
  'Rue des Vernes à Roanne\n\n\n\n',
  '#'],
 'para_embedding': array([-2.56996620e-02,  2.94211768e-02,  2.33180169e-02,  7.82801863e-03,
        -1.48487631e-02,  3.78930159e-02, -2.68976931e-02,  4.26564254e-02,
         5.15989289e-02, -7.38588581e-03, -4.01267707e-02, -1.80062763e-02,
         2.82343999e-02, -3.16207297e-02,  1.73622519e-02,  1.40902838e-02,
        -3.54822762e-02,  3.80511843e-02,  5.73955253e-02,  2.10540593e-02,
         1.22920386e-02,  2.19017286e-02,  7.41365105e-02,  5.48744798e-02,
         9.39616188e-03, -3.25019620e-02,  3.81131172e

In [9]:
clusters.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'])

In [10]:
clusters["cluster_0"].keys()

dict_keys(['para_indexes', 'union_paras'])

In [11]:
clusters["cluster_0"]

{'para_indexes': [1,
  10,
  14,
  19,
  30,
  31,
  33,
  37,
  40,
  46,
  52,
  57,
  61,
  64,
  69,
  72,
  73,
  77,
  80,
  84,
  85,
  86,
  89],
 'union_paras': "PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\n La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :\n\n\n\nChristine Aranéo - Marcel AugierDana Burnichon| | |Edmond Bourgeon| |\n\n|-| | |Catherine Brun| |\n\n|Jean-Marc Detour| | | | |\n\n|Daniel Fréchet| | |Lucien Murzi| |\n\n|Quentin Guillermin|X| | | |\n\n|Jean-Paul Heyberger| | |Christelle Lattat| |\n\n|Franck Maupetit| | |Annie Gerenton| |\n\n|Patrick Meunier|X| | | |\n\n|Nabih Nejjar|X| | |(arrivé en cours de séance)|\n\n|Gilles Passot| | |Fabien Lambert|(arrivé

Takes around 25 secs to run in my machine.

## Data Transformations

### Summarization algos

The process is fairly straightforward, we take the union of the paragraphs in each cluster and then we generate a summary for each cluster using gemma2-9b-it in groq. Given how slow the hf models we tried where, we will just use groq.


In [12]:
from council_rag.data_transformations.text_transformations import generate_groq_summary

In [13]:
summary_prompt = """Il s'agit d'un texte concernant un projet de géothermie :
TEXTE: 
{}

TÂCHE:
Make a one page summary paying special attention to all the administrative matters like budgets, plans, actions to take in the future, organizational and hierarchical charts, announcements, meetings, contacts, elections, reports and all the related topics to these.

OUTPUT:
Publier directement le résumé.
"""

In [17]:
cluster_paras = []
for cluster in clusters.values():
    cluster_para = cluster["union_paras"]
    cluster_paras.append(cluster_para)

print(len(cluster_paras))

8


gemma2-9b-it is the model with the largest Token limit per minute, 15000.

llama-3.3-70b-versatile allows 6000 per minute.

In [ ]:
import time
# max tokens for llama-3.3-70b-versatile is 6000 per minute
cluster_summaries = []

token_limit = 14000
token_count = 0
for cluster_para in cluster_paras:
    # print(len(cluster_para))
    naive_length_check = len(cluster_para.split(" ")) # the token calcs in groq are very close to this number
    token_count += naive_length_check
    # print(naive_length_check)
    if token_count > token_limit:
        time.sleep(60)
        token_count = 0
    cluster_summary = generate_groq_summary(summary_prompt, cluster_para, groq_token, model="gemma2-9b-it")
    cluster_summaries.append(cluster_summary)
    
# from 6 minutes to 28 secs in my machine

In [31]:
import json
with open("cluster_summaries_example.json", "w") as f:
    json.dump(cluster_summaries, f)


Making the defs for the utils

In [117]:
def summarize_clusters(cluster_paras, summary_prompt, groq_token, model="gemma2-9b-it", token_limit=14000):
    
    # cluster_summaries = []

    token_count = 0
    for cluster_para in cluster_paras:
        # print(len(cluster_para))
        naive_length_check = len(cluster_para.split(" ")) # the token calcs in groq are very close to this number
        token_count += naive_length_check
        # print(naive_length_check)
        if token_count > token_limit:
            time.sleep(60)
            token_count = 0
        cluster_summary = generate_groq_summary(summary_prompt, cluster_para, groq_token, model)
        
        cluster_para["clusteR_summary"] = cluster_summary
        # cluster_summaries.append(cluster_summary)
        
    return cluster_paras

## Table content algos

The process is a bit more complex. Since we want to put the augmented table context in the right place and with im2pdf we get the page where the table is, we have to estimate where in the running text the table is. A workaround could be extracting each page text of the pdf as markdown to keep track of what span of text happens in each page. Then we could just try to regex match the table context to the pages text and append the augmented table context above/below the table.

Note: some pdfs may have a page n/m at the end of the page but it is not always the case.


In [58]:
from council_rag.data_utils import pf2mdpagebypage, pdf2md


In [59]:
output_folder = "pdfs/"
input_pdf = "example.pdf"

In [54]:
# using our fx with fitz and the llama parse based fx
pages = pf2mdpagebypage(input_pdf=input_pdf, output_folder=output_folder)

# takes around 30 secs per page to run in my machine
# fifteen minutes for 20 pages
# 30 for the whole pdf

Started parsing the file under job_id e6c3be36-3b22-4c6f-a210-074b96ce3634
Started parsing the file under job_id ee3e530b-af77-4a45-af4b-1e21fc5c9721
..Started parsing the file under job_id 6817bf31-447b-4700-8725-f951bc060787
.Started parsing the file under job_id c8279860-22ea-49d6-8c26-eeceedcc2258
Started parsing the file under job_id 713a79d8-92a4-4e3f-aa92-dcce4cb73b56
Started parsing the file under job_id 35ca6f2c-6f9b-48fd-91c0-ad6285ed1fd1
Started parsing the file under job_id 2c74e8b3-7165-4797-aa23-7088a5930839
..Started parsing the file under job_id f8f06f75-18dd-44ea-8e63-4a33309666ed
.Started parsing the file under job_id 2b7159d8-9efb-4b7b-979c-fce21aef6c6a
Started parsing the file under job_id 4d706a17-5246-41f4-b1a7-421ef3bec0c9
Started parsing the file under job_id 35705776-5d03-4651-a335-0a830d3c19e5
Started parsing the file under job_id 90e34f19-364a-49e0-8db0-d2f5b17e6d86
Started parsing the file under job_id 8abd3a47-d709-4609-a73b-6035b4db7a98
Started parsing the

In [55]:
pages[0]

"# Séance publique\n\n# du jeudi 28 octobre 2021 à 18h00\n\n# Chorum Alain Gilles - Halle Vacheresse\n\n# Rue des Vernes à Roanne\n\n# PROCES VERBAL\n\nL'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l’Espace Chorum – Halle Vacheresse – Rue des Vernes à Roanne.\n\nLa convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n# Etaient présents :\n\nChristine Aranéo - Marcel Augier - Jean-Jacques Banchet - Pierre Barnet - Martine Barroso - Isabelle Berthelot -\nFranck Beysson - Jean-Yves Boire - Romain Bost - Michelle Bouchet - Edmond Bourgeon - Marie-Christine Bravo -\nCatherine Brun - Dominique Bruyère - Nicolas Chargueros - Jean-Luc Chervin - Christine Chevillard - Pierre Coissard -\nPatrick Collet - Sandra Creuzet - Hervé Daval - Jean-Paul Descombes - Pierre Devedeux - Christian Dorange -\nDavid Dozance - Catherine 

In [9]:
from council_rag.data_utils import pdf2md

In [10]:
import os
# can try more pdf mining algos here conviniently

list_pdfs = os.listdir("pdfs/")
list_image_pdfs = [pdf for pdf in list_pdfs if pdf.startswith("page")]

pages = []
for pdf in list_image_pdfs:
    txt = pdf2md("pdfs/" + pdf)
    pages.append(txt)

Started parsing the file under job_id b45c559b-51a8-4721-b5b8-cf3f77dd60e3
Started parsing the file under job_id 1351f1fe-4daa-4cb5-a85e-2d0934a311a5
Started parsing the file under job_id 56ad440c-fb71-4e1f-b9f9-fe663654de3a
.Started parsing the file under job_id 3e9158c6-d9fe-4f2d-b3af-5d84b3cdbf7b
Started parsing the file under job_id 8f75f722-5629-48f8-8923-fb43085a6395
Started parsing the file under job_id 06d2c580-26d3-4aa7-ac45-366a0630bbde
.Started parsing the file under job_id cae33628-d3ad-41b1-aed3-3bba49b67e3d
.Started parsing the file under job_id 924c24e7-3bd4-42f7-96cf-33b0ceeea5c0
.Started parsing the file under job_id d3412aa9-4d7a-45ec-abc9-037b6187fb6b
.Started parsing the file under job_id a753f25f-0f76-46fe-9090-53916e644b5f
..Started parsing the file under job_id d2c40d27-e168-4c69-8102-785f63b6bdce
.Started parsing the file under job_id e2952017-23f7-494d-bf6d-16979b7582dc
.Started parsing the file under job_id a0702d31-e903-4ec5-8a48-c7838bf70178
.Started parsing

In [12]:
import json

with open("pages.json", "w") as f:
    json.dump(pages ,f)

#### Matching the page to the content.

It is easy to see the results are not the same. We tried to match the content to page in a more rudimentary way but it did not work.
We followed this process:
try to get the nasty characters out and tokenize the alphanumerical characters so that we have normalized text. Then ask whether the string of consecutive characters are in each page number.

In [13]:
pages[0].replace("\n", "").replace("#", "").replace("  ", " ")

" Séance publique du jeudi 28 octobre 2021 à 18h00 Chorum Alain Gilles - Halle Vacheresse Rue des Vernes à Roanne PROCES VERBALL'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l’Espace Chorum – Halle Vacheresse – Rue des Vernes à Roanne.La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président. Etaient présents :Christine Aranéo - Marcel Augier - Jean-Jacques Banchet - Pierre Barnet - Martine Barroso - Isabelle Berthelot - Franck Beysson - Jean-Yves Boire - Romain Bost - Michelle Bouchet - Edmond Bourgeon - Marie-Christine Bravo - Catherine Brun - Dominique Bruyère - Nicolas Chargueros - Jean-Luc Chervin - Christine Chevillard - Pierre Coissard - Patrick Collet - Sandra Creuzet - Hervé Daval - Jean-Paul Descombes - Pierre Devedeux - Christian Dorange - David Dozance - Catherine Dufossé - Christian Dupuis - Jacky Geneste

In [14]:
paragraphs[0]["paragraph_union"].replace("\n", "").replace("#", "").replace("  ", " ")

' Séance publique du jeudi 28 octobre 2021 à 18h00 Chorum Alain Gilles - Halle Vacheresse Rue des Vernes à Roanne '

In [15]:
paragraphs[0]["paragraph_union"].replace("\n", "").replace("#", "").replace("  ", " ") in pages[0].replace("\n", "").replace("#", "")

True

In [16]:
clean_text = lambda x:x.replace("\n", "").replace("#", "").replace("-", " ").replace("  ", " ") # quick clean sentence processing

clean_text(paragraphs[0]["paragraph_union"]) in clean_text(pages[0]) # this would mean this sentence is in the first page

True

Recall paragraphs are groups of 10 sentences.

In [17]:
paragraphs[0].keys()

dict_keys(['paragraph_union', 'start_range', 'stop_range', 'list_sents', 'para_embedding', 'para_cluster'])

In [18]:
paragraphs[0]["stop_range"]

np.int64(10)

In [19]:
import spacy

nlp = spacy.load("fr_core_news_sm")

In [20]:
clean_pages = [clean_text(page) for page in pages] # a list of the clean text for every page
clean_pages_docs = [nlp(page) for page in clean_pages]
clean_page_text_tokens = []
for doc in clean_pages_docs:
    tokens = [token.text.lower() for token in doc if token.text.isalpha()] # filter nasty characters
    clean_page_text_tokens.append(tokens)

In [21]:
# clean_page_text_tokens[0]

In [22]:
for para in paragraphs:
    clean_para_text = clean_text(para["paragraph_union"])
    doc_clean_text = nlp(clean_para_text)
    clean_para_tokens = [token.text.lower() for token in doc_clean_text if token.text.isalpha()] # filter nasty characters
    para["clean_text"] = clean_para_tokens

In [23]:
# paragraphs[1]["clean_text"]

In [42]:
import numpy as np 

for para in paragraphs:
    para["page_n"] = None
    clean_para_tokens = para["clean_text"]
    i = 0
    agreement_array = []
    while i < len(clean_page_text_tokens):
        if " ".join(clean_para_tokens) in " ".join(clean_page_text_tokens[i]):
            para["page_n"] = i
            break
        
        i += 1    

In [44]:
# the fact that the markdowns are computed separately makes it so that the tokens in the text end up being different for the same span of text
for para in paragraphs:
    print(para["page_n"])

0
None
0
0
0
0
0
0
0
None
11
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
5
5
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Instead of injecting the tables in the text and just querying the text, we will input the table markdown with its description separately.

## Table context and description

In [58]:
from council_rag.data_transformations.table_transformations import extract_tables_from_page, convert_pdf_to_image, encode_image, format_multimodal_message_list, augment_multimodal_context

In [46]:
pdf_path = "example.pdf"
extracted_tables = extract_tables_from_page(pdf_path)

In [52]:
extracted_tables_filter = {k:v for k, v in extracted_tables.items() if v != []} # filtering the pages that actually have tables

In [55]:
list_pages_with_tables = list(extracted_tables_filter.keys())

In [72]:
image_paths = convert_pdf_to_image(pdf_path) # converting the pdf pages into images 

In [75]:
base_prompt="""CONTEXTE
L'image suivante contient une page et un tableau. 

TÂCHE
Décrivez le tableau et le contenu de la page en accordant une attention particulière au contexte qui l'entoure, qu'il s'agisse de budgets, de dates, d'élections, d'agendas, de projets futurs ou de sujets connexes. 

FORMAT DE LA RÉPONSE
Votre réponse doit être aussi détaillée que possible. 
Votre résultat doit être la description du tableau directement.
La langue de votre réponse est le français"""

In [77]:
table_descriptions = [augment_multimodal_context(im_path, base_prompt, groq_token) for im_path in image_paths]

In [79]:
with open("table_descr.json", "w") as f:
    json.dump(table_descriptions, f)

In [80]:
table_descriptions[:2]

['The image shows a French-language poster in which the public is informed about a conference held on October 28th, 2021 between 1 p.m. and 2 p.m. in Roannaise at the Heude Vacheresse site. Here are all the contents of the poster.\n\n*   **Banners and title banner**\n    *   The title banner, in place at the top of the page, is divided into two separate words. The left rectangle is black, while the right rectangle is an array of red letters. The text is in French. The text reads\nRoeauneis\nThe word "rois" contains a black circle with a white and red "PD" inside. The white and red letters are a symbol for the word "Roannaise," an agency supported by the French Republic.\n*   **Date of the event**\n    The poster includes the date of the meeting, which is stated without the year. In French, this is specifically mentioned as:\nOctober 28th, of year.\n*   **Meeting time**\n    The message informs the audience of an upcoming meeting set for October 28th, specifically on Thursday that last.

In [ ]:
# table_descriptions

In [96]:
list_table_objects = list(extracted_tables_filter.values())

In [89]:
# list_pages_with_tables

In [100]:
list_table_objects = list(extracted_tables_filter.values())
list_table_objects = [table[0] for table in list_table_objects] # unfolding the list of lists
list_table_htmls = [table.html for table in list_table_objects]

In [112]:
format_string = """{}
Tableau au format html:
{}"""

list_processed_tables = []
i = 0
while i < len(list_table_htmls):
    
    html_table = list_table_htmls[i]
    page_num = list_pages_with_tables[i]
    descr_table = table_descriptions
    table_string = format_string.format(descr_table, html_table)
    table_dict = {"page": page_num,
                "table_context": descr_table,
                "table_html": table_string,
                "table_augmented_context": table_string}
    list_processed_tables.append(table_dict)
    i += 1

In [114]:
# list_processed_tables[0:2]

Making the functions for the utils

In [115]:
def format_table_info(list_pages_with_tables, list_table_htmls, table_descriptions):
    
    format_string = """{}
Tableau au format html:
{}"""

    list_processed_tables = []
    i = 0
    while i < len(list_table_htmls):
        
        html_table = list_table_htmls[i]
        page_num = list_pages_with_tables[i]
        descr_table = table_descriptions
        table_string = format_string.format(descr_table, html_table)
        table_dict = {"page": page_num,
                    "table_context": descr_table,
                    "table_html": table_string,
                    "table_augmented_context": table_string}
        list_processed_tables.append(table_dict)
        i += 1
    return list_processed_tables

In [118]:
def process_tables(pdf_path, base_prompt, groq_token):

    extracted_tables = extract_tables_from_page(pdf_path)
    extracted_tables_filter = {k:v for k, v in extracted_tables.items() if v != []} # filtering the pages that actually have tables
    list_pages_with_tables = list(extracted_tables_filter.keys())
    
    # converting the pdf pages into images
    image_paths = convert_pdf_to_image(pdf_path)  
    table_descriptions = [augment_multimodal_context(im_path, base_prompt, groq_token) for im_path in image_paths]
    
    # getting table htmls
    list_table_objects = list(extracted_tables_filter.values())
    list_table_objects = [table[0] for table in list_table_objects] # unfolding the list of lists
    list_table_htmls = [table.html for table in list_table_objects]
    
    # processing them into a manageable format
    processed_tables = format_table_info(list_pages_with_tables, list_table_htmls, table_descriptions)
    
    return processed_tables